In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (c) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops


%matplotlib inline
np.random.seed(1)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
y_train = train['Survived']
print(train.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


# explore the data

In [4]:
y_train = train['Survived']
train = train.loc[:, train.columns != 'Survived']

In [58]:
from sklearn.model_selection import train_test_split
train, dev, y_train, y_dev = train_test_split(
    train.loc[:, train.columns != 'Survived'], train['Survived'], test_size=0.01, random_state=0)

In [59]:
train, dev

(     PassengerId  Pclass                                             Name  \
 367          368       3                   Moussa, Mrs. (Mantoura Boulos)   
 704          705       3                          Hansen, Mr. Henrik Juul   
 346          347       2                        Smith, Miss. Marion Elsie   
 196          197       3                              Mernagh, Mr. Robert   
 535          536       2                           Hart, Miss. Eva Miriam   
 ..           ...     ...                                              ...   
 835          836       1                      Compton, Miss. Sara Rebecca   
 192          193       3  Andersen-Jensen, Miss. Carla Christine Nielsine   
 629          630       3                         O'Connell, Mr. Patrick D   
 559          560       3     de Messemaeker, Mrs. Guillaume Joseph (Emma)   
 684          685       2                Brown, Mr. Thomas William Solomon   
 
         Sex   Age  SibSp  Parch        Ticket     Fare Cabin 

In [5]:
Y_train = y_train
Y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [6]:
def train_custom_linear_encoder(data):
    
    max_val = np.nanmax(data)
    min_val = np.nanmin(data)
    encoding_params = {'max_val':max_val,
                       'min_val':min_val}
    linear_encoded=(data-encoding_params['min_val'])/(
        encoding_params['max_val']-encoding_params['min_val'])
    
    linear_encoded = linear_encoded.values.reshape(-1,1)
    
    # Added a mask to deal with Nan Values
    mask=np.invert(np.isnan(linear_encoded))
    linear_encoded=np.nan_to_num(linear_encoded)
    
    # output
    return linear_encoded, mask, encoding_params

def test_custom_linear_encoder(data,encoding_params):
    linear_encoded=(data-encoding_params['min_val'])/(
        encoding_params['max_val']-encoding_params['min_val'])
    
    linear_encoded = linear_encoded.values.reshape(-1,1)
    
    # Added a mask to deal with Nan Values
    mask=np.invert(np.isnan(linear_encoded))
    linear_encoded=np.nan_to_num(linear_encoded)
    
    #output
    return linear_encoded, mask


array1, mask, encoding_params = train_custom_linear_encoder(train['Age'])
array2, mask = test_custom_linear_encoder(train['Age'], encoding_params)

assert np.all(np.equal(array1,array2,))

# integer encode
def train_custom_binary_encoder(data):
    unique_labels = data.unique()
    encoding_params = {}
    k=0
    for labels in unique_labels:
        encoding_params[labels] = k
        k=k+1
    encoding_params['other'] = k
    one_hot = np.zeros((data.shape[0],len(encoding_params)))
    mask = np.ones((data.shape[0],len(encoding_params)))
    for i, row in enumerate(data):
        one_hot[i,encoding_params[row]] = 1
    return one_hot, mask, encoding_params

def test_custom_binary_encoder(data, encoding_params):
    one_hot = np.zeros((data.shape[0],len(encoding_params)))
    mask = np.ones((data.shape[0],len(encoding_params)))
    for i, row in enumerate(data):
        try:
            one_hot[i,encoding_params[row]] = 1
        except:
            one_hot[i,encoding_params['other']] = 1
    return one_hot, mask

array1, mask, encoding_params = train_custom_binary_encoder(train['Embarked'])
array2, mask = test_custom_binary_encoder(train['Embarked'],encoding_params)
assert np.all(np.equal(array1,array2,))

In [7]:

['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

to_encode = {'Parch':'binary',
             'SibSp':'binary',
             'Embarked':'binary',
             'Pclass':'binary',
             'Sex':'binary',
             'Fare':'linear',
             'Age':'linear',
             'Cabin':'cabin'}

def train_pre_processor(pd_csv,to_encode):
    """
    arguments
        inputs -- straight from the input
        
    returns
        outputs -- arguments to train
    """
    encoders = []
    encoded_data = []
    encoder_mask = []
    for encoder_key in to_encode.keys():
        encoder_type = to_encode[encoder_key]
        if encoder_type == 'binary':
            encoded_col, mask, encoding_params = train_custom_binary_encoder(pd_csv[encoder_key])
        elif encoder_type == 'linear':
            encoded_col, mask, encoding_params = train_custom_linear_encoder(pd_csv[encoder_key])
        elif encoder_type == 'cabin':
            deck = pd_csv[encoder_key].str.slice(0,1)
            encoded_col, mask, encoding_params = train_custom_binary_encoder(deck)
        else:
            encoded_col = np.reshape(pd_csv[encoder_key],(-1,1))
            mask = np.ones((encoded_col.shape))
            encoding_params = np.nan
        encoders.append(encoding_params)
        encoded_data.append(encoded_col)
        encoder_mask.append(mask)

    inputs_processed= np.concatenate(encoded_data,axis=1)
    mask_processed = np.concatenate(encoder_mask,axis=1)
    output = np.array([inputs_processed, mask_processed])
    output = tf.transpose(output, perm=[1,0,2])
    
    return output, encoders

X_train, encoders = train_pre_processor(train, to_encode)
print(X_train.shape)
print(encoders)


def test_pre_processor(pd_csv,encoders, to_encode):
    """
    arguments
        inputs -- straight from the input
        
    returns
        outputs -- arguments to train
    """
    encoded_data = []
    encoder_mask = []
    k=0
    for i, encoder_key in enumerate(to_encode.keys()):
        encoder_type = to_encode[encoder_key]
        encoding_params = encoders[i]
        if encoder_type == 'binary':
            encoded_col, mask = test_custom_binary_encoder(pd_csv[encoder_key],encoding_params)
        elif encoder_type == 'linear':
            encoded_col, mask = test_custom_linear_encoder(pd_csv[encoder_key],encoding_params)
        elif encoder_type == 'cabin':
            deck = pd_csv[encoder_key].str.slice(0,1)
            encoded_col, mask = test_custom_binary_encoder(deck,encoding_params)
        else:
            encoded_col = np.reshape(pd_csv[encoder_key],(-1,1))
            mask = np.ones((encoded_col.shape))
            encoding_params = np.nan
        encoded_data.append(encoded_col)
        encoder_mask.append(mask)
        

    inputs_processed= np.concatenate(encoded_data,axis=1)
    mask_processed = np.concatenate(encoder_mask,axis=1)
    output = np.array([inputs_processed, mask_processed])
    output = tf.transpose(output, perm=[1,0,2])
    return output



X_test = test_pre_processor(test, encoders, to_encode)
print(X_test.shape)

(891, 2, 40)
[{0: 0, 1: 1, 2: 2, 5: 3, 3: 4, 4: 5, 6: 6, 'other': 7}, {1: 0, 0: 1, 3: 2, 4: 3, 2: 4, 5: 5, 8: 6, 'other': 7}, {'S': 0, 'C': 1, 'Q': 2, nan: 3, 'other': 4}, {3: 0, 1: 1, 2: 2, 'other': 3}, {'male': 0, 'female': 1, 'other': 2}, {'max_val': 512.3292, 'min_val': 0.0}, {'max_val': 80.0, 'min_val': 0.42}, {nan: 0, 'C': 1, 'E': 2, 'G': 3, 'D': 4, 'A': 5, 'B': 6, 'F': 7, 'T': 8, 'other': 9}]
(418, 2, 40)


2023-11-06 11:01:13.416890: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-06 11:01:13.416917: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-11-06 11:01:13.416923: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-11-06 11:01:13.417140: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-06 11:01:13.417166: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
def oneLayerNN(a):
    """
    Simple one layer neural network
    
    Arguments:
    None

    Returns:
    model -- TF Keras model 
    """
    shape_ = a.shape
    model = tf.keras.Sequential([
            tfl.Input(shape=(shape_[1],shape_[2])),
            tfl.Dense(50, activation='relu'),
            tfl.Flatten(),
            tfl.Dense(50, activation='relu'),
            tfl.Dense(1, activation='sigmoid'),
        ])
    
    return model

In [21]:
one_layer_nn = oneLayerNN(X_train)
one_layer_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                   loss=tf.keras.losses.BinaryCrossentropy(),
                   metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])

In [22]:
one_layer_nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2, 50)             2050      
                                                                 
 flatten_2 (Flatten)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 7151 (27.93 KB)
Trainable params: 7151 (27.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
one_layer_nn.fit(X_train, Y_train, epochs=200)

Epoch 1/200
 1/28 [>.............................] - ETA: 11s - loss: 0.7259 - binary_accuracy: 0.4062 - false_negatives_2: 5.0000

2023-11-06 11:06:28.078953: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


28/28 [==============================] - 1s 13ms/step - loss: 0.6633 - binary_accuracy: 0.6173 - false_negatives_2: 314.0000
Epoch 2/200
28/28 [==============================] - 0s 9ms/step - loss: 0.5792 - binary_accuracy: 0.7194 - false_negatives_2: 241.0000
Epoch 3/200
28/28 [==============================] - 0s 10ms/step - loss: 0.5007 - binary_accuracy: 0.7823 - false_negatives_2: 130.0000
Epoch 4/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4664 - binary_accuracy: 0.7912 - false_negatives_2: 113.0000
Epoch 5/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4574 - binary_accuracy: 0.8025 - false_negatives_2: 110.0000
Epoch 6/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4405 - binary_accuracy: 0.8081 - false_negatives_2: 98.0000
Epoch 7/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4379 - binary_accuracy: 0.8114 - false_negatives_2: 95.0000
Epoch 8/200
28/28 [==============================] - 0s 9ms/

In [29]:
X_dev = test_pre_processor(dev, encoders, to_encode)
Y_dev = one_layer_nn(X_dev)
y_devhat = Y_dev>0.5
y_devhat = y_devhat.numpy()
dev.shape

NameError: name 'dev' is not defined

In [30]:
# np.equal(y_dev,y_devhat)
series = np.equal(np.reshape(y_dev.to_numpy(),(-1,1)),y_devhat)
np.sum(series)/len(series)

NameError: name 'y_dev' is not defined

In [31]:
X_test = test_pre_processor(test, encoders,to_encode)
X_test = one_layer_nn(X_test)
xresult = X_test>0.5
x_result= xresult.numpy()
x_result.shape

(418, 1)

In [32]:
pass_id = test['PassengerId'].to_numpy()
pass_col=pass_id.reshape(-1,1)
pass_col.shape

(418, 1)

In [33]:
import datetime
data_result = np.concatenate((pass_col,x_result),axis=1)
submission = pd.DataFrame(data_result,columns=["PassengerId","Survived"])
submission.to_csv('submission.csv',index=False)
print(f"file saved at {datetime.datetime.now()}")

file saved at 2023-11-06 11:07:47.396452
